<a href="https://colab.research.google.com/github/GEORMC/Nnumerical_Methods_Course/blob/main/3Nodes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

def triangular_element_stiffness(E, nu, coords):
    """
    Calculate the stiffness matrix for a triangular finite element.

    Parameters:
        E (float): Young's modulus
        nu (float): Poisson's ratio
        coords (ndarray): Coordinates of the three vertices, a 3x2 array

    Returns:
        K (ndarray): 6x6 stiffness matrix of the triangular element
    """
    # Area of the triangle
    x1, y1 = coords[0]
    x2, y2 = coords[1]
    x3, y3 = coords[2]
    A = 0.5 * abs((x2 - x1) * (y3 - y1) - (x3 - x1) * (y2 - y1))

    # B-matrix components
    b = np.array([y2 - y3, y3 - y1, y1 - y2])
    c = np.array([x3 - x2, x1 - x3, x2 - x1])

    # Strain-displacement matrix B
    B = np.zeros((3, 6))
    for i in range(3):
        B[0, 2 * i] = b[i]
        B[1, 2 * i + 1] = c[i]
        B[2, 2 * i] = c[i]
        B[2, 2 * i + 1] = b[i]
    B /= (2 * A)

    # Elasticity matrix D for plane stress or plane strain
    factor = E / (1 - nu ** 2)
    D = factor * np.array([[1, nu, 0],
                           [nu, 1, 0],
                           [0, 0, (1 - nu) / 2]])

    # Stiffness matrix K
    K = A * B.T @ D @ B

    return K

def apply_boundary_conditions(K, F, fixed_dofs):
    """
    Modify the stiffness matrix and force vector to apply boundary conditions.

    Parameters:
        K (ndarray): Global stiffness matrix
        F (ndarray): Global force vector
        fixed_dofs (list): List of indices where displacements are zero (fixed DOFs)

    Returns:
        K_mod (ndarray): Modified stiffness matrix
        F_mod (ndarray): Modified force vector
    """
    # Modify the stiffness matrix and force vector for boundary conditions
    for dof in fixed_dofs:
        K[dof, :] = 0
        K[:, dof] = 0
        K[dof, dof] = 1
        F[dof] = 0
    return K, F

def solve_displacements(K, F):
    """
    Solve for the nodal displacements.

    Parameters:
        K (ndarray): Stiffness matrix
        F (ndarray): Force vector

    Returns:
        displacements (ndarray): Nodal displacements
    """
    displacements = np.linalg.solve(K, F)
    return displacements

# Material properties and coordinates
E = 210e9  # Young's modulus in Pascals
nu = 0.3   # Poisson's ratio
coords = np.array([[0, 0], [1, 0], [0, 1]])  # Coordinates of triangle vertices

# Element stiffness matrix
K_element = triangular_element_stiffness(E, nu, coords)

# Global stiffness matrix (for a single element, this is just K_element)
K_global = K_element.copy()

# External force vector (6 DOFs, 2 per node for a triangle)
F_global = np.zeros(6)
F_global[1] = -1000  # Apply a downward force of 1000 N at node 1 (index 1)

# Boundary conditions (fix the x and y displacements at nodes 0 and 2)
fixed_dofs = [0, 2, 3, 5]  # x-y displacements of nodes 0 and 2 are fixed

# Apply boundary conditions
K_global_bc, F_global_bc = apply_boundary_conditions(K_global, F_global, fixed_dofs)

# Solve for displacements
displacements = solve_displacements(K_global_bc, F_global_bc)

# Output the results
print("Nodal Displacements:\n", displacements)
